# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to Jack parent
import os
os.chdir('..')

In [2]:
# Configure Jack logger to print to standard output, that is console
import sys, os
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger(os.path.basename(sys.argv[0]))

### Bookkeeping of all existing readers: `readers.py`

In [3]:
import jtr.jack.readers as readers
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
example_reader, modelf_reader, snli_reader, fastqa_reader


### Load test data

In [4]:
# Loaded some test data to work on
# This loads train, dev, and test data of sizes (2k, 1k, 1k)
from jtr.jack.core import TestDatasets
train_set, dev_set, test_set = TestDatasets.generate_SNLI()

### Create a reader

In [5]:
from jtr.preprocess.vocab import Vocab
# Create example reader with a basic config
embedding_dim = 128
hidden_dim = 128
config = {"batch_size": 128, "repr_dim": hidden_dim, "repr_dim_input": embedding_dim, 'dropout' : 0.1}
reader = readers.readers["snli_reader"](Vocab(), config)

### Add hooks

In [6]:
# We creates hooks which keep track of the loss
# We also create 'the standard hook' for our model
from jtr.jack.train.hooks import LossHook, ClassificationEvalHook
hooks = [LossHook(reader, iter_interval=10),
         readers.eval_hooks['snli_reader'](reader, dev_set, iter_interval=25)]

### Initialize optimizer

In [7]:
# Here we initialize our optimizer
# we choose Adam with standard momentum values and learning rate 0.001
import tensorflow as tf
learning_rate = 0.001
optim = tf.train.AdamOptimizer(learning_rate)

### Train reader

In [ ]:
# Lets train the reader on the CPU for 2 epochs
reader.train(optim, train_set,
             hooks=hooks, max_epochs=2,
             device='/cpu:0')

INFO:root:Setting up data and model...
INFO:root:Start training...
INFO:jtr.jack.train.hooks:Epoch 1	Iter 10	Loss 1.10627046824
INFO:jtr.jack.train.hooks:Epoch 1	Iter 20	Loss 1.11009055376
INFO:jtr.jack.train.hooks:Started evaluation ClassificationEvalHook
INFO:jtr.jack.train.hooks:Epoch 1	Iter 25	total 1000	exact: 0.329	f1: 0.000	ClassificationEvalHook
INFO:jtr.jack.train.hooks:Epoch 1	Iter 30	Loss 1.10287501812
INFO:jtr.jack.train.hooks:Epoch 1	Iter 40	Loss 1.10451259613
INFO:jtr.jack.train.hooks:Epoch 1	Iter 50	Loss 1.10709911585
INFO:jtr.jack.train.hooks:Started evaluation ClassificationEvalHook
INFO:jtr.jack.train.hooks:Epoch 1	Iter 50	total 1000	exact: 0.359	f1: 0.000	ClassificationEvalHook
